In [21]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, math, time
#aditional imports

In [22]:
# Load API Credentials
with open('/Users/alisonwilliams/Documents/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [23]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [24]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE_pizza = "Data/results_in_progress_NY_pizza.json"
JSON_FILE_pizza


'Data/results_in_progress_NY_pizza.json'

In [25]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE_pizza)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE_pizza)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE_pizza} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE_pizza,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE_pizza} already exists.")

[i] Data/results_in_progress_NY_pizza.json already exists.


In [26]:
## Load previous results and use len of results for offset
with open(JSON_FILE_pizza,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 1000 previous results found.


In [27]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [ ]:
## How many results total?
total_results = results['total']
total_results


In [28]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [29]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


695

In [30]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE_pizza,'w') as f:
     json.dump(previous_results,f)

In [31]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/695 [00:00<?, ?it/s]

In [32]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE_pizza, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE_pizza,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


  0%|          | 0/695 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: 1020 is greater than the maximum of 1000

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE_pizza, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE_pizza,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE_pizza)
display(final_df.head(), final_df.tail())

In [ ]:
# check for duplicate results
final_df.duplicated().sum()


In [ ]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()


In [ ]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)
